# ----------------------------------MISSING DATA----------------------------------------

### Imports

In [2]:
import pandas as pd
import numpy as np
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing,neighbors
import os

### Reading each CSV in a separate dataframe

In [5]:
directory = os.path.dirname(os.getcwd()) 
newpathloanFinal = directory+'/Dataset/Loan'
d1 = pd.read_csv(newpathloanFinal+'/LoanStats3a_securev1.csv', header=1, skipfooter=4, engine='python')
d2 = pd.read_csv(newpathloanFinal+'/LoanStats3b_securev1.csv', header=1, skipfooter=4, engine='python')
d3 = pd.read_csv(newpathloanFinal+'/LoanStats3c_securev1.csv', header=1, skipfooter=4, engine='python')
d4 = pd.read_csv(newpathloanFinal+'/LoanStats3d_securev1.csv', header=1, skipfooter=4, engine='python')
d5 = pd.read_csv(newpathloanFinal+'/LoanStats_securev1_2016Q1.csv', header=1, skipfooter=4, engine='python')
d6 = pd.read_csv(newpathloanFinal+'/LoanStats_securev1_2016Q2.csv', header=1, skipfooter=4, engine='python')
d7 = pd.read_csv(newpathloanFinal+'/LoanStats_securev1_2016Q3.csv', header=1, skipfooter=4, engine='python')
d8 = pd.read_csv(newpathloanFinal+'/LoanStats_securev1_2016Q4.csv', header=1, skipfooter=4, engine='python')
d9 = pd.read_csv(newpathloanFinal+'/LoanStats_securev1_2017Q1.csv', header=1, skipfooter=4, engine='python')
d10 = pd.read_csv(newpathloanFinal+'/LoanStats_securev1_2017Q2.csv', header=1, skipfooter=4, engine='python')
d11 = pd.read_csv(newpathloanFinal+'/LoanStats_securev1_2017Q3.csv', header=1, skipfooter=4, engine='python')

### Concatenating all the CSVs in one dataframe

In [6]:
loan_data = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10,d11])

In [7]:
loan_df = loan_data

In [8]:
loan_df.shape

(1646773, 151)

In [9]:
loan_df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=151)

### Finding the missing data percentage and storing columns in features with mores than 70% data

In [10]:
missing_data = loan_df.isnull().sum().to_frame(name='Missing_Count').reset_index()
missing_data['Missing_percent'] = (missing_data['Missing_Count']/loan_data.shape[0])*100
missing_data.head()
missing_data = missing_data.sort_values(by='Missing_percent', axis=0, ascending=True)
features = missing_data['index'][(missing_data.Missing_percent <30)]
print('Number of columns after removing missing columns -',len(features))

Number of columns after removing missing columns - 93


In [11]:
loan_df = loan_df[features]

In [12]:
del loan_df['id']
loan_df = loan_df.reset_index()
loan_df['index'] = loan_df.index +1
loan_df=loan_df.rename(columns = {'index':'id'})

### Considering only the features that are relevant to analysis

In [13]:
f = ['id','debt_settlement_flag','application_type','fico_range_low','fico_range_high','emp_length','dti','annual_inc','grade','sub_grade','int_rate','loan_amnt','issue_d','purpose','addr_state','home_ownership','zip_code','policy_code','term']

In [14]:
loan_df= loan_df[f]

In [15]:
loan_df.shape

(1646773, 19)

In [16]:
loan_df = loan_df[loan_df.addr_state!='WV']
loan_df = loan_df[loan_df.addr_state!='IA']

In [17]:
loan_df.isnull().sum()

id                        0
debt_settlement_flag      1
application_type          1
fico_range_low            1
fico_range_high           1
emp_length                1
dti                     358
annual_inc                5
grade                     1
sub_grade                 1
int_rate                  1
loan_amnt                 1
issue_d                   1
purpose                   1
addr_state                1
home_ownership            1
zip_code                  2
policy_code               1
term                      1
dtype: int64

In [18]:
loan_df = loan_df[loan_df.application_type.notnull()]

### Emp Length

In [19]:
loan_df['emp_length'].replace(to_replace='[^0-9]+', value='',inplace=True, regex=True)
loan_df.loc[loan_df.emp_length=='', ['emp_length']] = '0'
loan_df['emp_length'] = loan_df['emp_length'].astype(int)

### Zipcode

In [20]:
loan_df['zip_code'] = loan_df['zip_code'].astype(str).map(lambda x: x.rstrip('xx'))

In [21]:
loan_df = loan_df[loan_df.zip_code!='nan']

In [22]:
z = loan_df[['zip_code', 'addr_state']][loan_df.addr_state=='AL'].groupby(['addr_state']).apply(pd.DataFrame.mode).reset_index(drop=True)['zip_code'][0]
z = int(z)
loan_df.loc[loan_df.zip_code=='nan', ['zip_code']] =z

In [23]:
loan_df['zip_code'] = loan_df['zip_code'].astype(int)

### Int Rate

In [24]:
loan_df['int_rate'] = loan_df['int_rate'].astype(str).map(lambda x: x.rstrip('%'))
loan_df['int_rate'] = loan_df['int_rate'].astype(float)

### Date

In [25]:
loan_df['Year'] = loan_df['issue_d'].astype(str).map(lambda x: int(x.split('-')[1]))
loan_df['Month'] = loan_df['issue_d'].astype(str).map(lambda x: x.split('-')[0])

In [26]:
lookup = {'Dec':12, 'Nov':11, 'Oct':10, 'Sep':9, 'Aug':8, 'Jul':7, 'Jun':6, 'May':5, 'Apr':4,'Mar':3, 'Feb':2, 'Jan':1}
loan_df['Month'] =  loan_df['Month'].apply(lambda x: lookup[x])

In [27]:
loan_df[['Year','Month']] = loan_df[['Year','Month']].astype(int)

In [28]:
# Annual Income
part = loan_df[['id','emp_length', 'annual_inc']]
Mode = loan_df['annual_inc'].mode()[0]
a = list(loan_df['emp_length'].unique())

for c in a:
    t = part[part.emp_length == c]
    if( t.annual_inc.isnull().sum()==0):
            break
    else:
            if(t.shape == t[t.isnull().any(axis=1)].shape ):
                t['annual_inc'].fillna(Mode, inplace=True)
            else:
                c = t['annual_inc'].mode()[0]
                t['annual_inc'].fillna(c, inplace=True)
            part.loc[part.Id.isin(t.id), ['annual_inc']] = t[['annual_inc']]
part['annual_inc'].fillna(part['annual_inc'].mode()[0], inplace = True)      
loan_df.loc[loan_df.id.isin(part.id), ['annual_inc']] = part[['annual_inc']]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


### dti

In [29]:
loan_df = loan_df[loan_df.dti.notnull()]
loan_df.head()

,id,debt_settlement_flag,application_type,fico_range_low,fico_range_high,emp_length,dti,annual_inc,grade,sub_grade,...,loan_amnt,issue_d,purpose,addr_state,home_ownership,zip_code,policy_code,term,Year,Month
0,1,N,Individual,735.0,739.0,10,27.65,24000.0,B,B2,...,5000.0,Dec-2011,credit_card,AZ,RENT,860,1.0,36 months,2011,12
1,2,N,Individual,740.0,744.0,1,1.00,30000.0,C,C4,...,2500.0,Dec-2011,car,GA,RENT,309,1.0,60 months,2011,12
2,3,N,Individual,735.0,739.0,10,8.72,12252.0,C,C5,...,2400.0,Dec-2011,small_business,IL,RENT,606,1.0,36 months,2011,12
3,4,N,Individual,690.0,694.0,10,20.00,49200.0,C,C1,...,10000.0,Dec-2011,other,CA,RENT,917,1.0,36 months,2011,12
4,5,N,Individual,695.0,699.0,1,17.94,80000.0,B,B5,...,3000.0,Dec-2011,other,OR,RENT,972,1.0,60 months,2011,12


In [30]:
loan_df = loan_df[loan_df['annual_inc'].notnull()]

In [31]:
# dti
dti_t = loan_df[['dti', 'emp_length','id']]
Mode_dti = loan_df['dti'].mode()[0]
d = list(loan_df['emp_length'].unique())

for c in d:
    dt = dti_t[dti_t.emp_length == c]
    if( dt.dti.isnull().sum()==0):
            break
    else:
            if(dt.shape == dt[dt.isnull().any(axis=1)].shape ):
                dt['dti'].fillna(Mode_dti, inplace=True)
            else:
                c = dt['dti'].mode()[0]
                dt['dti'].fillna(c, inplace=True)
            dti_t.loc[dti_t.id.isin(dt.id), ['dti']] = dt[['dti']]
dti_t['dti'].fillna(dti_t['dti'].mode()[0], inplace = True)      
loan_df.loc[loan_df.id.isin(dti_t.id), ['dti']] = dti_t[['dti']]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


### Fico columns

In [32]:
loan_df['fico_avg'] = (loan_df['fico_range_high']+loan_df['fico_range_low'])/2

In [33]:
loan_df['term'] = loan_df['term'].astype(str).map(lambda x: x.rstrip('months'))
loan_df['term'] = loan_df['term'].astype(int)

In [34]:
loan_df =loan_df[loan_df.dti>-1]

In [35]:
loan_df.shape

(1641242, 22)

In [36]:
loan_df = loan_df[loan_df.fico_avg > 660]

In [37]:
loan_df[['grade','sub_grade','application_type','addr_state','purpose','home_ownership']]= loan_df[['grade','sub_grade','application_type','addr_state','purpose','home_ownership']].astype(str)

In [38]:
loan_df['policy_code'] = loan_df['policy_code'].astype(int)

In [39]:
loan_df.dtypes

id                        int64
debt_settlement_flag     object
application_type         object
fico_range_low          float64
fico_range_high         float64
emp_length                int32
dti                     float64
annual_inc              float64
grade                    object
sub_grade                object
int_rate                float64
loan_amnt               float64
issue_d                  object
purpose                  object
addr_state               object
home_ownership           object
zip_code                  int32
policy_code               int32
term                      int32
Year                      int32
Month                     int32
fico_avg                float64
dtype: object

### Exporting the cleaned data into a CSV

In [40]:
loan_df['approval'] =1
loan_df=loan_df.rename(columns = {'fico_avg':'fico'})
loan_df=loan_df.rename(columns = {'addr_state':'State'})

In [43]:
loan_df.to_csv('/Users/madhu/Desktop/ADS Project/Dataset/Loan/Loan.csv', index=False)

In [42]:
loan_df.shape

(1640757, 23)

The text in the document by SHREYANTH REDDY BEZAWADA is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/

Copyright 2018 SHREYANTH REDDY BEZAWADA AND ADITYA MOHAN KUMAR

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.